## Prerequisites

In [1]:
!pip install numpy matplotlib pandas sklearn

In [2]:
!python -V

Python 3.8.8


## Data Preprocessing

In [4]:
from scipy.io import loadmat

In [5]:
import pickle

In [6]:
import h5py

In [7]:
data = loadmat(r'ST640-GaN-Si-50-generate TTR not shift.mat')

In [1]:
import scipy

In [2]:
X = data['re']

NameError: name 'data' is not defined

In [10]:
X.shape

(125000, 300)

In [12]:
import pickle
import numpy as np

In [13]:
np.set_printoptions(suppress=True)

In [14]:
X = np.transpose(data['re'])

In [15]:
X.shape

(300, 125000)

In [16]:
X = X.T

In [17]:
X.shape

(125000, 300)

In [18]:
X = X[:,20:299]

In [20]:
X.shape

(125000, 279)

In [30]:
layer_2 = np.linspace(10, 250, 50, endpoint=True)
layer_3 = np.linspace(20, 200, 50, endpoint=True)
layer_4 = np.linspace(50, 250, 50, endpoint=True)

In [33]:
y = np.zeros((X.shape[0], 3))

In [34]:
i = 0
for l4 in layer_4:
    for l3 in layer_3:
        for l2 in layer_2:
            y[i] = [l4, l3, l2]
            i = i + 1

In [35]:
y.shape

(125000, 3)

### Train test split

In [318]:
from sklearn.model_selection import train_test_split

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [320]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [321]:
#property = 0

In [322]:
x_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

x_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [323]:
# Builds dataset with ALL data
origin_train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(origin_train_dataset, [int(x_train_tensor.shape[0] * 0.9), int(x_train_tensor.shape[0] * 0.1)])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=2000)
val_loader = DataLoader(dataset=val_dataset, batch_size=2000)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader  = DataLoader(dataset=test_dataset, batch_size=2000)

## Model

In [324]:
import torch.nn as nn

In [325]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm1d(X.shape[1])
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        return x

## Training

In [326]:
import torch.optim as optim

In [327]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [397]:
n_epochs = 80

In [398]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yh = model(x)
        #yh = torch.reshape(yh, (-1,))
        loss = loss_fn(y, yh)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [399]:
model = Net().to(device)

loss_fn = nn.MSELoss(reduction='mean')

# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

train_step = make_train_step(model, loss_fn, optimizer)

In [400]:
model.eval()

Net(
  (bn1): BatchNorm1d(279, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=279, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [401]:
training_losses = []
validation_losses = []

for epoch in range(n_epochs):
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yh = model(x_val)
            #yh = torch.reshape(yh, (-1,))
            val_loss = loss_fn(y_val, yh).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.5f}\t Validation loss: {validation_loss:.5f}")

[1] Training loss: 20686.91609	 Validation loss: 20327.73926
[2] Training loss: 20117.33214	 Validation loss: 19649.30729
[3] Training loss: 19327.03552	 Validation loss: 18712.97038
[4] Training loss: 18254.93327	 Validation loss: 17498.78385
[5] Training loss: 16918.61514	 Validation loss: 16030.13688
[6] Training loss: 15433.56386	 Validation loss: 14579.24593
[7] Training loss: 13937.99161	 Validation loss: 13114.66960
[8] Training loss: 12418.70258	 Validation loss: 11712.78499
[9] Training loss: 11132.27228	 Validation loss: 10621.22705
[10] Training loss: 10156.11350	 Validation loss: 9704.74235
[11] Training loss: 9434.24398	 Validation loss: 9150.21305
[12] Training loss: 8960.26666	 Validation loss: 8707.86507
[13] Training loss: 8666.42636	 Validation loss: 8478.22005
[14] Training loss: 8517.48297	 Validation loss: 8374.77539
[15] Training loss: 8432.60787	 Validation loss: 8330.54093
[16] Training loss: 8386.04621	 Validation loss: 8272.25903
[17] Training loss: 8336.53800

In [402]:
# model.state_dict()

## Testing

In [403]:
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [404]:
x_test_tensor = x_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
y_pred = model(x_test_tensor).squeeze()

In [405]:
test_loss = loss_fn(y_test_tensor, y_pred)
print(test_loss)

tensor(21.1191, grad_fn=<MseLossBackward0>)


In [406]:
print(f"The mean of absolute percentage error: {mean_absolute_percentage_error(y_test_tensor.cpu(), y_pred.cpu()):.2f}%")

The mean of absolute percentage error for L1: 3.85%


## Real-world case

In [99]:
import pandas as pd

In [100]:
case1 = pd.read_csv('20230220036.csv', header=None)

In [103]:
case1.shape

(1, 300)

In [104]:
case1= np.array(case1)
case1=case1[:,20:299]
case1_tensor = torch.from_numpy(case1).float()

In [409]:
model.eval()
pred_1 = model(case1_tensor)

In [410]:
pred_1

tensor([[171.9459,  70.7058, 134.6853]], grad_fn=<ReluBackward0>)

# 保存模型

In [422]:
torch.save(model,'GaN-Si140-50^3-12-8000-BATCH2000-EPO80-lr0.01-20230729.pth')

In [413]:
AU_SIMODEL = torch.load('GaN-SiC300-12.5w-108-299-BATCH2000-EPO100-lr0.005-20230316.pth')

FileNotFoundError: [Errno 2] No such file or directory: 'GaN-SiC300-12.5w-108-299-BATCH2000-EPO100-lr0.005-20230316.pth'

In [ ]:
case1 = pd.read_csv('GaN160+GaN100+TBC1+SiC300.csv', header=None)

In [486]:
pred_2 = AU_SIMODEL(case1_tensor)

In [487]:
pred_2

tensor([[0.5689],
        [0.5230],
        [0.5014],
        [0.3863],
        [0.5495]], grad_fn=<ReluBackward0>)

In [ ]:
Au-Gan-SiC100-6-1500-generate TTR not shift